# Overview

The purpose of this repository is to implement advanced controls algorithms and simulate their usage on example systems. At the moment, the repository has an implementation of an Extended Kalman Filter and Model Predictive Controller, as well as a nonlinear model of an inverted pendulum on a cart. This document includes some derivations of the implemented algorithms, mostly because it is helpful to my own understanding to derive them myself instead of just reading the final equations from a paper. Sources are cited where needed. Also included in this document are results from the simulations using these algorithms.

# Linearization

Please refer to [https://www.cds.caltech.edu/~murray/courses/cds101/fa02/caltech/pph02-ch19-23.pdf](https://www.cds.caltech.edu/~murray/courses/cds101/fa02/caltech/pph02-ch19-23.pdf) for a very thorough treatment of linearizing a nonlinear system model.

# Model Predictive Control

## Quadratic Programming Formulation

Assume we have a discrete time system of the form:

$$ x[k+1] = Ax[k] + Bu[k] $$
$$ y[k] = Cx[k] $$

The goal of Model Predictive Control is to determine the optimal input sequence, $u_k, u_{k+1}, ... , u_{k+L-1}$ that minimizes a specified cost function. For mathematical convenience, a quadratic cost function is often chosen, because it leads to a form that is easy to solve using standard optimization techniques. Below is the quadratic cost function for a reference tracking Model Predictive Controller:

$$ J = {(x[k+L] - x_{ref}[k+L])}^TQ_{f}(x[k+L] - x_{ref}[k+L]) + \sum_{i=k}^{k+L-1} (x[i] - x_{ref}[i])^TQ(x[i] - x_{ref}[i]) + {u[i]}^TRu[i]$$

where $x_{ref}$ is the desired state trajectory for the system, $L$ is the length of the prediction horizon, $Q$ is the state cost matrix, $Q_f$ is the terminal state cost matrix, and $R$ is the input cost matrix. For reasons that will become clear later in the derivation, we require that $Q$ and $Q_{f}$ are positive semidefinite, $R$ is positive definite, and all three are symmetric.

(Note that we've ignored specifying a $u_{ref}$ signal that will drive the system to follow $x_{ref}$ - that is because it is often difficult to know in advance what the reference input should be. This can lead to steady state error in the controller, but this can be addressed by adding integral action to compensate for the error. This will be discussed in a later section, see the section on Reference Tracking and Integral Action in [https://www.kth.se/social/upload/5194b53af276547cb18f4624/lec13_mpc2_4up.pdf](https://www.kth.se/social/upload/5194b53af276547cb18f4624/lec13_mpc2_4up.pdf) for more information).

The equation above is a little messy to look at and it isn't in a form that is easy to work with. Let's simplify it by first observing that the system state evolves as follows:

$$ x[k+1] = Ax[k] + Bu[k] $$
$$ x[k+2] = A^2x[k] + ABu[k] + Bu[k+1] $$
$$ . $$
$$ . $$
$$ . $$
$$ x[k+L] = A^{L}x[k] + A^{L-1}Bu[k] + A^{L-2}Bu[k+2] + ... + ABu[k+L-2] + Bu[k+L-1] $$

Let's define new "state", "input", and "reference" vectors, $\hat{X}$, $\hat{U}$, and $\hat{X}_{ref}$ that look like:

$$ \hat{X} = \begin{bmatrix} x[k+1] \\ x[k+2] \\ ... \\ x[k+L] \end{bmatrix}, \hat{U} = \begin{bmatrix} u[k] \\ u[k+1] \\ ... \\ u[k+L-1] \end{bmatrix}, \hat{X}_{ref} = \begin{bmatrix} x_{ref}[k+1] \\ x_{ref}[k+2] \\ ... \\ x_{ref}[k+L] \end{bmatrix} $$

Following the pattern that we see above in the state evolution equations, we can write the equation for $\hat{X}$ in matrix form:

$$ 
\hat{X} = 
\begin{bmatrix}
A   \\ 
A^2 \\
A^3 \\
... \\
A^L
\end{bmatrix} x[k] + 
\begin{bmatrix}
B    & 0   &   0 & ... & 0 \\
AB   & B   &   0 & ... & 0 \\
A^2B & AB  &   B & ... & 0 \\
...  & ... & ... & ... & 0 \\
A^{L-1}B & A^{L-2}B & A^{L-3}B &  ... & B
\end{bmatrix} \hat{U}
$$
$$ \hat{X} = \hat{A}x[k] + \hat{B}\hat{U} $$

We'll similarly define new weight matrices $\hat{Q}$ and $\hat{R}$:

$$ \hat{Q} = \begin{bmatrix}
Q & 0 & 0  & ... & 0 \\
0 & Q & 0  & ... & 0 \\
0 & 0 & Q  & ... & 0 \\
0 & 0 & 0  & ... & Q_f \\
\end{bmatrix},
\hat{R} = \begin{bmatrix}
R & 0 & 0  & ... & 0 \\
0 & R & 0  & ... & 0 \\
0 & 0 & R  & ... & 0 \\
0 & 0 & 0  & ... & R \\
\end{bmatrix}
$$

Using these new matrices, you can convince yourself that we can re-write and simplify the original cost function as follows:

$$ J = (\hat{A}x[k] + \hat{B}\hat{U} - \hat{X}_{ref})^T\hat{Q}(\hat{A}x[k] + \hat{B}\hat{U} - \hat{X}_{ref}) + \hat{U}\hat{R}\hat{U} $$
$$ J = ({x[k]}^T\hat{A}^T\hat{Q} + \hat{U}^T\hat{B}^T\hat{Q} - \hat{X}_{ref}^T\hat{Q})(\hat{A}x[k] + \hat{B}\hat{U} - \hat{X}_{ref}) + \hat{U}\hat{R}\hat{U} $$
$$ 
J = {x[k]}^T\hat{A}^T\hat{Q}\hat{A}x[k] + {x[k]}^T\hat{A}^T\hat{Q}\hat{B}\hat{U} - {x[k]}^T\hat{A}^T\hat{Q}\hat{X}_{ref} + 
\hat{U}^T\hat{B}^T\hat{Q}\hat{A}x[k] + \hat{U}^T\hat{B}^T\hat{Q}\hat{B}\hat{U} - \hat{U}^T\hat{B}^T\hat{Q}\hat{X}_{ref} - 
\hat{X}_{ref}^T\hat{Q}\hat{A}x[k] - \hat{X}_{ref}^T\hat{Q}\hat{B}\hat{U} + \hat{X}_{ref}^T\hat{Q}\hat{X}_{ref} + \hat{U}\hat{R}\hat{U}
$$

Recall that the goal is to find the value of $\hat{U}$ the minimizes $J$, and not the minimum value of $J$ itself. As a result, we can omit all constant terms; that is, all that do not depend on $\hat{U}$.

$$ J = {x[k]}^T\hat{A}^T\hat{Q}\hat{B}\hat{U} + \hat{U}^T\hat{B}^T\hat{Q}\hat{A}x[k] + \hat{U}^T\hat{B}^T\hat{Q}\hat{B}\hat{U} - \hat{U}^T\hat{B}^T\hat{Q}\hat{X}_{ref} - \hat{X}_{ref}^T\hat{Q}\hat{B}\hat{U} + \hat{U}\hat{R}\hat{U} $$

Our next trick is to realize that since J is a scalar function, each term in the above equation is a scalar, so we can freely transpose any of the terms without changing the expression. Let's transpose all of the non-quadratic terms starting with $\hat{U}^T$.

$$ J = {x[k]}^T\hat{A}^T\hat{Q}\hat{B}\hat{U} + {x[k]}^T\hat{A}^T\hat{Q}^TBU + \hat{U}^T\hat{B}^T\hat{Q}\hat{B}\hat{U} - \hat{X}_{ref}^T\hat{Q}^T\hat{B}\hat{U} - \hat{X}_{ref}^T\hat{Q}\hat{B}\hat{U} + \hat{U}\hat{R}\hat{U} $$

And for our final trick, recall that we require $\hat{Q}$ to be symmetric, so we can replace $\hat{Q}^T$ with $\hat{Q}$.

$$ J = {x[k]}^T\hat{A}^T\hat{Q}\hat{B}\hat{U} + {x[k]}^T\hat{A}^T\hat{Q}BU + \hat{U}^T\hat{B}^T\hat{Q}\hat{B}\hat{U} - \hat{X}_{ref}^T\hat{Q}\hat{B}\hat{U} - \hat{X}_{ref}^T\hat{Q}\hat{B}\hat{U} + \hat{U}\hat{R}\hat{U} $$
$$ J = \hat{U}^T(\hat{B}^T\hat{Q}\hat{B} + \hat{R})\hat{U} + 2({x[k]}^T\hat{A}^T\hat{Q}\hat{B} - \hat{X}_{ref}^T\hat{Q}\hat{B})\hat{U} $$

The form of the final, simplified cost equation has a special form, $J = \hat{U}^TH\hat{U} + 2f^TU$, which is known as a quadratic form. We can now easily use any off the shelf quadratic programming solver to minimize the equation. QP solvers also allow adding equality and inequality constraints, which are useful for constraining the state and input variables to their physical limits. The current implemenation doesn't allow specifying such constraints, but that is something that will be added soon.

This derivation in this section is mostly based on the one found in [https://arxiv.org/pdf/2109.11986.pdf](https://arxiv.org/pdf/2109.11986.pdf). The main difference is that this one omits including the initial state in $\hat{X}$, which in my opinion simplifies the analysis. The derivation in that document also skips some important algebra and doesn't explain why $Q$ and $Q_f$ (and as a result, $\hat{Q}$) must be symmetric matrices for the math to work out properly, which is why I've re-done the derivation here.

## Terminal Constraints

In order for MPC to be stable (at least when constraints are not present / activated), we must be very particular about the terminal state cost matrix, $Q_f$, is chosen. Specifically, it is the solution to the discrete algebraic Riccati equation, as specified in [https://www.mathworks.com/help/mpc/ug/terminal-weights-and-constraints.html](https://www.mathworks.com/help/mpc/ug/terminal-weights-and-constraints.html). Based on actually simulating the results without using a terminal cost matrix, I can confirm that the controller is indeed unstable unless an unreasonably long prediction horizon is used.

# Extended Kalman Filter

Coming soon!

# Inverted Pendulum Simulation

Please refer to [https://ctms.engin.umich.edu/CTMS/index.php?example=InvertedPendulum&section=SystemModeling](https://ctms.engin.umich.edu/CTMS/index.php?example=InvertedPendulum&section=SystemModeling) for a derivation of the equations of motion for the inverted pendulum system dynamics. For this simulation, the goal was to use an EKF to estimate the state of the system, and MPC as the controller. The full nonlinear dynamics of the model were simulated to obtain the actual state of the cart. It also included a Gaussian noise term to reflect reality. The desired trajectory for the system was to have the cart follow a sinusoidally varying position, while maintaining the pendulum in an upright position. Two different formulations for MPC were used, one where the state is augmented with an input integrator and the other without. As shown in the plots below, both schemes performed about equally, though the one using integral action generated a much smoother input signal, which is desirable for real systems.

<img src="README_RES/IP_MPC_Integral_Action.png"/>

<img src="README_RES/IP_MPC_No_Integral_Action.png"/>